In [2]:
import pandas as pd
import numpy as np
movies=pd.read_csv('ml-100k/movies.csv')
links=pd.read_csv('ml-100k/links.csv')
ratings=pd.read_csv('ml-100k/ratings.csv')
tags=pd.read_csv('ml-100k/tags.csv')

In [3]:
# int(movies.movieId[movies.title=='Toy Story (1995)'])
movies.iloc[0].title

'Toy Story (1995)'

In [4]:
def strrep(genre):
    return genre.replace('|',' ')

In [5]:
# x='Adventure|Animation|Children|Comedy|Fantasy'
# x.replace('|',' ')
movies['genres']=movies.genres.apply(strrep)

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure Drama Romance
9121,163056,Shin Godzilla (2016),Action Adventure Fantasy Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
genresvect=vectorizer.fit_transform(movies['genres'])

genresvect.shape

(9125, 23)

In [8]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(genresvect, genresvect)

In [9]:
cosine_sim

array([[1.        , 0.80403568, 0.15641664, ..., 0.        , 0.26646851,
        0.        ],
       [0.80403568, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15641664, 0.        , 1.        , ..., 0.        , 0.58699859,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        1.        ],
       [0.26646851, 0.        , 0.58699859, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        1.        ]])

In [10]:
indices=pd.Series(movies.index,index=movies['title']).drop_duplicates()
indices

title
Toy Story (1995)                                                0
Jumanji (1995)                                                  1
Grumpier Old Men (1995)                                         2
Waiting to Exhale (1995)                                        3
Father of the Bride Part II (1995)                              4
                                                             ... 
Mohenjo Daro (2016)                                          9120
Shin Godzilla (2016)                                         9121
The Beatles: Eight Days a Week - The Touring Years (2016)    9122
The Gay Desperado (1936)                                     9123
Women of '69, Unboxed                                        9124
Length: 9125, dtype: int64

In [11]:
def get_recommendation(title,cosine_sim=cosine_sim):
    idx=indices[title]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]
    print(sim_scores)

    movie_indices = [i[0] for i in sim_scores]

    return movies['title'].iloc[movie_indices]

In [12]:
get_recommendation('Toy Story (1995)')

[(1815, 0.9999999999999999), (2506, 0.9999999999999999), (3003, 0.9999999999999999), (3217, 0.9999999999999999), (3805, 0.9999999999999999), (6705, 0.9999999999999999), (7146, 0.9999999999999999), (7945, 0.9999999999999999), (8366, 0.9999999999999999), (8676, 0.9999999999999999)]


1815                                          Antz (1998)
2506                                   Toy Story 2 (1999)
3003       Adventures of Rocky and Bullwinkle, The (2000)
3217                     Emperor's New Groove, The (2000)
3805                                Monsters, Inc. (2001)
6705                               Shrek the Third (2007)
7146                       Tale of Despereaux, The (2008)
7945    Asterix and the Vikings (Astérix et les Viking...
8366                                         Turbo (2013)
8676                                Boxtrolls, The (2014)
Name: title, dtype: object

In [13]:
def add_user_ratings(mid, rat, ratings):
    new_user = pd.DataFrame({'userId': [672]*10,
                             'movieId': mid,
                             'rating': rat
                             })
    ratings=ratings.append(new_user)
    return ratings
mid=[]
rat=[]
for i in range(10):
    mid.append(input('Enter movie id'))
    rat.append(input('Enter rating'))
ratings=add_user_ratings(mid, rat, ratings)

Enter movie id6268
Enter rating4
Enter movie id31
Enter rating3
Enter movie id6385
Enter rating5
Enter movie id1029
Enter rating2
Enter movie id1815
Enter rating1
Enter movie id3003
Enter rating5
Enter movie id6705
Enter rating3
Enter movie id3805
Enter rating5
Enter movie id3217
Enter rating2
Enter movie id1172
Enter rating4


C:\Program Files\Anaconda\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [14]:

ratings


,movieId,rating,timestamp,userId
0,31,2.5,1.260759e+09,1
1,1029,3,1.260759e+09,1
2,1061,3,1.260759e+09,1
3,1129,2,1.260759e+09,1
4,1172,4,1.260759e+09,1
...,...,...,...,...
5,3003,5,NaN,672
6,6705,3,NaN,672
7,3805,5,NaN,672
8,3217,2,NaN,672


In [15]:
from surprise import SVD
from surprise import Dataset,Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))

data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']],reader)
trainset, testset = train_test_split(data, test_size=.25)

svd = SVD()

svd.fit(trainset)
predictions = svd.test(testset)

accuracy.rmse(predictions)

RMSE: 0.9010


0.9009501339306846

In [16]:
# pred = svd.test(testset)
predictions

[Prediction(uid=466, iid=1270, r_ui=3.0, est=4.145015429345903, details={'was_impossible': False}),
 Prediction(uid=242, iid=1266, r_ui=5.0, est=4.505924336929988, details={'was_impossible': False}),
 Prediction(uid=250, iid=31685, r_ui=4.5, est=3.6706797467914365, details={'was_impossible': False}),
 Prediction(uid=21, iid=1088, r_ui=2.0, est=3.389240312538016, details={'was_impossible': False}),
 Prediction(uid=119, iid=1350, r_ui=3.0, est=3.438584592890534, details={'was_impossible': False}),
 Prediction(uid=236, iid=1208, r_ui=5.0, est=4.0073837450101175, details={'was_impossible': False}),
 Prediction(uid=342, iid=2871, r_ui=4.0, est=4.383760221280405, details={'was_impossible': False}),
 Prediction(uid=285, iid=2404, r_ui=2.0, est=2.4439711158694966, details={'was_impossible': False}),
 Prediction(uid=41, iid=748, r_ui=3.5, est=3.6600374875517483, details={'was_impossible': False}),
 Prediction(uid=505, iid=2100, r_ui=3.0, est=2.909121343734413, details={'was_impossible': False})

In [18]:
svd.predict(672,11)#(userid,movieid)

Prediction(uid=672, iid=11, r_ui=None, est=3.690832348985759, details={'was_impossible': False})

In [19]:
from surprise import KNNBasic
knn=KNNBasic()
knn.fit(trainset)
predknn=knn.test(testset)
accuracy.rmse(predknn)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9733


0.9733364841703178

In [20]:
ratings[ratings['userId'] == 1]

,movieId,rating,timestamp,userId
0,31,2.5,1.260759e+09,1
1,1029,3,1.260759e+09,1
2,1061,3,1.260759e+09,1
3,1129,2,1.260759e+09,1
4,1172,4,1.260759e+09,1
5,1263,2,1.260759e+09,1
6,1287,2,1.260759e+09,1
7,1293,2,1.260759e+09,1
8,1339,3.5,1.260759e+09,1
9,1343,2,1.260759e+09,1


In [21]:
knn.predict(1, 50)#(userid,movieid)

Prediction(uid=1, iid=50, r_ui=None, est=4.362218655082403, details={'actual_k': 40, 'was_impossible': False})

In [22]:
from surprise import NMF

nmf = NMF()

nmf.fit(trainset)
predictions = nmf.test(testset)

accuracy.rmse(predictions)

RMSE: 0.9469


0.9468770139292285

In [23]:
from surprise import KNNWithMeans
knnm=KNNWithMeans()
knnm.fit(trainset)
predknn=knnm.test(testset)
accuracy.rmse(predknn)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9215


0.9215448262850093

In [24]:
def hybrid(userid,title):
    idx=indices[title]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in sim_scores]
    
    movie=movies.iloc[movie_indices]
    movie['est']=movie['movieId'].apply(lambda x: svd.predict(userid,x).est)
    movie = movie.sort_values('est', ascending=False)
    return movie.head(10)

In [25]:
hybrid(60,'Toy Story (1995)')

C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,movieId,title,genres,est
3805,4886,"Monsters, Inc. (2001)",Adventure Animation Children Comedy Fantasy,4.320761
2506,3114,Toy Story 2 (1999),Adventure Animation Children Comedy Fantasy,4.088591
3217,4016,"Emperor's New Groove, The (2000)",Adventure Animation Children Comedy Fantasy,3.980388
7945,91355,Asterix and the Vikings (Astérix et les Viking...,Adventure Animation Children Comedy Fantasy,3.960947
7146,65577,"Tale of Despereaux, The (2008)",Adventure Animation Children Comedy Fantasy,3.885321
8676,114552,"Boxtrolls, The (2014)",Adventure Animation Children Comedy Fantasy,3.863552
8366,103755,Turbo (2013),Adventure Animation Children Comedy Fantasy,3.821818
1815,2294,Antz (1998),Adventure Animation Children Comedy Fantasy,3.808009
6705,53121,Shrek the Third (2007),Adventure Animation Children Comedy Fantasy,3.772371
3003,3754,"Adventures of Rocky and Bullwinkle, The (2000)",Adventure Animation Children Comedy Fantasy,3.763585


In [26]:
#only collabrative filtering
movies['est']=movies['movieId'].apply(lambda x: svd.predict(672,x).est)
movies = movies.sort_values('est', ascending=False)
movies.head(10)

,movieId,title,genres,est
733,912,Casablanca (1942),Drama Romance,4.493494
472,527,Schindler's List (1993),Drama War,4.453606
284,318,"Shawshank Redemption, The (1994)",Crime Drama,4.393620
786,969,"African Queen, The (1951)",Adventure Comedy Romance War,4.393282
725,904,Rear Window (1954),Mystery Thriller,4.363950
746,926,All About Eve (1950),Drama,4.321548
2764,3462,Modern Times (1936),Comedy Drama Romance,4.306230
984,1228,Raging Bull (1980),Drama,4.281807
960,1203,12 Angry Men (1957),Drama,4.266737
968,1212,"Third Man, The (1949)",Film-Noir Mystery Thriller,4.265734
